# Overview

This notebook will import event logs from SDP pipelines that are not configured to store the event log
directly into a Delta table (see the [`event_log` option](https://docs.databricks.com/api/workspace/pipelines/create#event_log) in the
Pipelines API). This can happen, for example, if the pipeline was created prior to the introduction of ability to [Publish to Multiple Catalogs and Schemas from a Single DLT/SDP Pipeline](https://www.databricks.com/blog/publish-multiple-catalogs-and-schemas-single-dlt-pipeline).


The logs will be imported into the Delta table `{monitoring_catalog}.{monitoring_schema}.{imported_event_logs_table}`. This enables incremental processing of these logs and also access from principals that are not the owners of these pipelines.

Note that the import of such event logs uses a `MERGE` statement (to allow for incremental import) which is a fairly expensive operation. The preferred approach is to configure the event log into a Delta table.

# Parameters
- `monitoring_catalog` - the catalog for the table with the imported logs.
- `monitoring_schema` - the schema for the table with the imported logs
- `imported_event_logs_table_name` - the name for the table with the imported events logs
- `imported_pipeline_ids` - a comma-separated list of pipelines whose event logs are to be imported
- `imported_pipeline_tags` - Semi-colon-separated lists of comma-separated `tag[:value]` pairs. See the documentation for the `imported_pipeline_tags` variable in `vars/import_event_logs.vars.yml` for more information.

In [ ]:
import logging
import sys

sys.path.append("../lib")

from databricks_ingestion_monitoring.common import EventLogImporter

dbutils.widgets.text("monitoring_catalog", "")
dbutils.widgets.text("monitoring_schema", "")
dbutils.widgets.text("imported_event_logs_table_name", "imported_event_logs")
dbutils.widgets.text("imported_pipeline_ids", "")
dbutils.widgets.text("imported_pipeline_tags", "")
dbutils.widgets.text("pipeline_tags_index_table_name", "pipeline_tags_index")
dbutils.widgets.text("pipeline_tags_index_enabled", "true")
dbutils.widgets.text("pipeline_tags_index_max_age_hours", "24")
dbutils.widgets.text("pipeline_tags_index_api_fallback_enabled", "true")

logging.basicConfig(level=logging.INFO, format="%(asctime)s [%(levelname)s] (%(name)s) %(message)s")

importer = EventLogImporter(
    monitoring_catalog=dbutils.widgets.get("monitoring_catalog"),
    monitoring_schema=dbutils.widgets.get("monitoring_schema"),
    imported_event_logs_table=dbutils.widgets.get("imported_event_logs_table_name"),
    index_table_name=dbutils.widgets.get("pipeline_tags_index_table_name"),
    index_enabled=dbutils.widgets.get("pipeline_tags_index_enabled").lower() == "true",
    index_max_age_hours=int(dbutils.widgets.get("pipeline_tags_index_max_age_hours")),
    api_fallback_enabled=dbutils.widgets.get("pipeline_tags_index_api_fallback_enabled").lower() == "true"
)
importer.import_event_logs_for_pipelines_by_ids_and_tags(
    dbutils.widgets.get("imported_pipeline_ids"),
    dbutils.widgets.get("imported_pipeline_tags"),
    spark)